In [1]:
pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 90.3 MB/s 
     |████████████████████████████████| 981 kB 53.8 MB/s 
     |████████████████████████████████| 265 kB 70.1 MB/s 
     |████████████████████████████████| 3.8 MB 65.4 MB/s 
     |████████████████████████████████| 1.3 MB 71.5 MB/s 
     |████████████████████████████████| 468 kB 61.7 MB/s 
     |████████████████████████████████| 880 kB 69.5 MB/s 
     |████████████████████████████████| 182 kB 74.7 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.32.3-py3-none-any.whl size=25313585 sha256=be0e59b2c3b5f5916955af89703fcb075d3c23ae81f444833f0d685e596118ab
  Stored in directory: /root/.cache/pip/wheels/c4/a0/4b/38365824295b72c59f340fce7927cdb5438d2705f5dbbe47b7
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=c04e05106046babaf9f249f

In [2]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [4]:
data_train = pd.read_csv('sample_data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('sample_data/data_test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...
5,sadness,When my family heard that my Mother's cousin w...
6,joy,Finding out I am chosen to collect norms for C...
7,anger,A spokesperson said : ` Glen is furious that t...
8,neutral,Yes .
9,sadness,"When I see people with burns I feel sad, actua..."


In [5]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [6]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [7]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [8]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [9]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1323/1323 [==============================] - 833s 614ms/step - loss: 0.9057 - accuracy: 0.6528 - val_loss: 0.5839 - val_accuracy: 0.7907
Epoch 2/3
1323/1323 [==============================] - 810s 612ms/step - loss: 0.4360 - accuracy: 0.8547 - val_loss: 0.5596 - val_accuracy: 0.8093
Epoch 3/3
1323/1323 [==============================] - 809s 611ms/step - loss: 0.1795 - accuracy: 0.9448 - val_loss: 0.5771 - val_accuracy: 0.8302


In [10]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

107/107 [==============================] - 104s 923ms/step
              precision    recall  f1-score   support

         joy       0.87      0.84      0.85       707
     sadness       0.83      0.80      0.81       676
        fear       0.87      0.87      0.87       679
       anger       0.80      0.81      0.80       693
     neutral       0.79      0.84      0.81       638

    accuracy                           0.83      3393
   macro avg       0.83      0.83      0.83      3393
weighted avg       0.83      0.83      0.83      3393



array([[592,  16,  17,  15,  67],
       [ 15, 542,  34,  56,  29],
       [ 17,  20, 590,  40,  12],
       [ 17,  53,  28, 559,  36],
       [ 39,  25,  13,  27, 534]])

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [16]:
import time 

message = 'love you'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

1/1 [==============================] - 0s 71ms/step
predicted: joy (0.13)


In [15]:
# let's save the predictor for later use
predictor.save("sample_data/bert_model")